In [5]:
from langchain_groq import ChatGroq

In [6]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='', 
    model_name="llama-3.1-70b-versatile"
)

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.naukri.com/job-listings-cognizant-hiring-for-net-react-developer-cognizant-technology-solutions-india-ltd-kolkata-6-to-11-years-211124502745?src=jobsearchDesk&sid=17354079545416691&xp=1&px=1")
data = loader.load().pop().page_content
print(data)

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {job_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'job_data':data})

In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

[{'role': 'Software Engineer', 'experience': '2-5 years', 'skills': ['Python', 'Java', 'C++'], 'description': 'Design, develop, and test software applications'}, {'role': 'Data Scientist', 'experience': '5-10 years', 'skills': ['Machine Learning', 'Data Analysis', 'Statistics'], 'description': 'Analyze and interpret complex data to inform business decisions'}, {'role': 'Product Manager', 'experience': '5-10 years', 'skills': ['Product Development', 'Marketing', 'Leadership'], 'description': 'Lead the development and launch of new products'}]


In [ ]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df